In [17]:
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd


In [1]:
polygon_file = "isochrones.gpkg"
polygons = gpd.read_file(polygon_file)

In [103]:
izo30 = polygons[polygons['iso_time'] == 30]
izo15 = polygons[polygons['iso_time'] == 15]
izo10 = polygons[polygons['iso_time'] == 10]
izo5 = polygons[polygons['iso_time'] == 5]
izo3 = polygons[polygons['iso_time'] == 3]

In [104]:
train = pd.read_csv('train.csv')
train['type'] = 'train'
test = pd.read_csv('test.csv')
test['type'] = 'test'
data = pd.concat([train, test], ignore_index=True)

geometry = [Point(xy) for xy in zip(data['x'], data['y'])]
shops = gpd.GeoDataFrame(data, geometry=geometry, crs="EPSG:4326")

In [105]:
len(data)

26017

In [106]:
for num, izo in zip([3, 5, 10, 15, 30], [izo3, izo5, izo10, izo15, izo30]):
    points_in_polygons = gpd.sjoin(shops, izo, how="inner", op="within")
    points_count_per_polygon = points_in_polygons.groupby('id_left').size()
    points_count_per_polygon.name = 'count_pvz_iso_' + str(num)
    points_count_per_polygon.fillna(1, inplace=True)
    data = data.merge(points_count_per_polygon, how='left', left_on='id', right_index=True)

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3263: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):
/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3263: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):
/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3263: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):
/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3263: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use th

In [107]:
len(data)

26017

In [108]:
for num in [3, 5, 10, 15, 30]:
    data['count_pvz_iso_' + str(num)].fillna(1, inplace = True)
    data['pop_per_pvz_iso' + str(num)] = data['people_iso_' + str(num)] / data['count_pvz_iso_' + str(num)]

In [109]:
new_train = data[data['type'] == 'train']
new_test = data[data['type'] == 'test']

new_train.to_csv('train_pop_per_pvz_by_iso.csv', index = False)
new_test.to_csv('test_pop_per_pvz_by_iso.csv', index = False)

In [110]:
data.corr()

,id,sales_per_month,x,y,people_iso_3,people_iso_5,people_iso_10,people_iso_15,people_iso_30,count_pvz_iso_3,count_pvz_iso_5,count_pvz_iso_10,count_pvz_iso_15,count_pvz_iso_30,pop_per_pvz_iso3,pop_per_pvz_iso5,pop_per_pvz_iso10,pop_per_pvz_iso15,pop_per_pvz_iso30
id,1.000000,-0.004944,0.001232,-0.004413,0.002397,0.004325,0.002537,-0.000991,0.002668,0.018766,0.008268,-0.004404,-0.004547,0.002329,-0.000638,-0.000482,0.008382,0.001179,-0.002775
sales_per_month,-0.004944,1.000000,0.088249,0.047394,0.075708,0.116139,0.113531,0.081762,0.047567,-0.063785,-0.078565,-0.085698,-0.055305,-0.022394,0.085468,0.160372,0.245213,0.224988,0.164875
x,0.001232,0.088249,1.000000,-0.022070,-0.091295,-0.101890,-0.088304,-0.075664,-0.072814,-0.046080,-0.095807,-0.150806,-0.159436,-0.164715,-0.084978,-0.063738,0.052132,0.110500,0.139352
y,-0.004413,0.047394,-0.022070,1.000000,0.140009,0.178255,0.189887,0.176998,0.150108,0.016421,0.039591,0.110668,0.125871,0.122528,0.141836,0.183508,0.116367,0.041210,-0.049673
people_iso_3,0.002397,0.075708,-0.091295,0.140009,1.000000,0.739681,0.562955,0.497656,0.438394,0.156336,0.298292,0.421011,0.434725,0.416551,0.978097,0.646648,0.267922,0.145612,0.039849
people_iso_5,0.004325,0.116139,-0.101890,0.178255,0.739681,1.000000,0.795702,0.695209,0.585224,0.176207,0.382954,0.563790,0.581872,0.544163,0.719952,0.882176,0.408084,0.229999,0.072539
people_iso_10,0.002537,0.113531,-0.088304,0.189887,0.562955,0.795702,1.000000,0.932804,0.776820,0.127124,0.324516,0.664605,0.735187,0.684902,0.553206,0.710873,0.543123,0.337888,0.136544
people_iso_15,-0.000991,0.081762,-0.075664,0.176998,0.497656,0.695209,0.932804,1.000000,0.880378,0.100325,0.261436,0.613975,0.767925,0.758850,0.492222,0.639028,0.523625,0.371517,0.162721
people_iso_30,0.002668,0.047567,-0.072814,0.150108,0.438394,0.585224,0.776820,0.880378,1.000000,0.068848,0.190775,0.483803,0.659242,0.847884,0.438824,0.558489,0.481351,0.357558,0.187222
count_pvz_iso_3,0.018766,-0.063785,-0.046080,0.016421,0.156336,0.176207,0.127124,0.100325,0.068848,1.000000,0.475028,0.249099,0.197814,0.141685,0.000587,-0.013766,-0.054815,-0.062013,-0.062597


In [114]:
points_in_polygons = gpd.sjoin(shops, izo15, how="inner", op="within")
points_count_per_polygon = points_in_polygons.groupby('id_left').size()
points_count_per_polygon.name = 'count_pvz_iso_' + str(num)
points_count_per_polygon.fillna(1, inplace=True)
data = data.merge(points_count_per_polygon, how='left', left_on='id', right_index=True)

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3263: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


In [115]:
points_count_per_polygon

id_left
1        1
2        4
3        3
4        2
5        7
        ..
26013    2
26014    3
26015    2
26016    6
26017    1
Name: count_pvz_iso_30, Length: 26016, dtype: int64